<a href="https://colab.research.google.com/github/crlsyajie/Google/blob/main/Utilize_Embedding_in_ADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilize ADK to use a Gemini Embedding
####by Joan Santoso

This notebook demonstrates how to integrate the Agent Development Kit (ADK) with Gemini Embeddings for a Retrieval Augmented Generation (RAG) system.

1.  **Initialize Gemini Embedding:** The `client.models.embed_content` function is used with the `gemini-embedding-001` model to generate embeddings for a collection of documents. This turns human-readable text into dense vector representations.

2.  **FAISS Indexing:** These document embeddings are then used to create a FAISS (Facebook AI Similarity Search) index. FAISS is an efficient library for similarity search, allowing for fast retrieval of relevant documents based on their vector representations.

3.  **ADK Agent with Tool:** An `LlmAgent` from ADK is created, configured to use a Gemini model (e.g., `gemini-2.0-flash`). A custom tool, `faiss_search`, is provided to this agent. This tool takes a user query, embeds it using Gemini, searches the FAISS index for similar documents, and returns the retrieved text.

4.  **Instruction-based Retrieval:** The ADK agent is given an instruction: "If the user asks a question that requires knowledge about large language models, vector databases, or AI development, use the 'faiss_search' tool..." This enables the agent to intelligently decide when to use the `faiss_search` tool to augment its responses with external knowledge, effectively implementing a RAG pattern.


#### Step 1: Install Required Libraries
First, you need to install the necessary Python packages. `google-adk` is Google's Agent Development Kit, which provides the framework for building AI agents. `faiss-cpu` is Facebook AI Similarity Search library, an efficient library for similarity searching and clustering of dense vectors.

```python
!pip install google-adk faiss-cpu
```

In [ ]:
!pip install google-adk faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 16.3 MB/s eta 0:00:00


#### Step 2: Initialize Gemini Client and Define Document Collection
To interact with Gemini models for embedding, you need to set up the Gemini client. This involves importing the google genai library and providing your `GOOGLE_API_KEY`. It's recommended to store your API key securely using Colab's secrets manager. Also, define the set of documents that your RAG system will query.

In [ ]:
class DocumentDB:
    def __init__(self):
        self.docs_to_embed = [
            "Gemini is a powerful family of large language models.",
            "The Agent Development Kit (ADK) simplifies building multi-agent applications.",
            "FAISS is an open-source library for efficient similarity search.",
            "A vector database stores embeddings and allows for fast similarity queries.",
            "Retrieval Augmented Generation (RAG) improves LLM responses with external knowledge.",
            "The Eiffel Tower is in Paris, France."
        ]
        self.index_to_text_map = {i: text for i, text in enumerate(self.docs_to_embed)}

#### Step 3: Generate Embeddings for Documents
Embeddings are numerical representations of text that capture its semantic meaning. The `gemini-embedding-001` model is used here to convert each document in your `DocumentDB` into a vector. The `RETRIEVAL_DOCUMENT` task type specifies that these embeddings will be used for document retrieval.

In [ ]:
from google import genai
from google.genai import types
import os
import faiss
import numpy as np
from google.colab import userdata
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = "FALSE"
os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
DIMENSION = 768
API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=API_KEY)

In [ ]:
collection = DocumentDB()

results = client.models.embed_content(
    model="models/gemini-embedding-001",
    contents=collection.docs_to_embed,
    config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT", output_dimensionality=DIMENSION)
)


#### Step 4: Create and Populate a FAISS Index
FAISS (Facebook AI Similarity Search) is used to create an efficient index for your document embeddings. This index allows for very fast nearest neighbor searches. An `IndexFlatL2` index uses Euclidean distance (L2 norm) to measure similarity. The embeddings are converted to a NumPy array of `float32` type before being added to the index.

In [ ]:
document_embeddings = [emb.values for emb in results.embeddings]
document_embeddings_np = np.array(document_embeddings).astype('float32')
index = faiss.IndexFlatL2(DIMENSION)
index.add(document_embeddings_np)
print(f"FAISS index created with {index.ntotal} vectors.")
faiss.write_index(index, "vector.db")

FAISS index created with 6 vectors.



#### Step 5: Define a Tool for FAISS Search
To allow the ADK agent to interact with the FAISS index, you need to define a Python function that acts as a tool. This `faiss_search` tool takes a user query, embeds it (using `RETRIEVAL_QUERY` task type), performs a similarity search on the FAISS index, and returns the text of the most relevant documents.


In [ ]:
def faiss_search(query: str, k: int = 1) -> str:
    """
    Searches the FAISS index for the most similar documents to a given query.
    """
    # 1. Embed the user's query
    query_embedding_response = client.models.embed_content(
        model="models/gemini-embedding-001",
        contents=query,
        config=types.EmbedContentConfig(task_type="RETRIEVAL_QUERY", output_dimensionality=DIMENSION)
    )
    query_embedding = np.array([query_embedding_response.embeddings[0].values]).astype('float32')

    # 2. Perform the similarity search in the FAISS index
    distances, indices = index.search(query_embedding, k)

    # 3. Retrieve the corresponding documents and format the result
    retrieved_docs = []
    for i in indices[0]:
        retrieved_docs.append(collection.index_to_text_map[i])

    result_text = "Retrieved documents:\n" + "\n".join(
        [f"- {doc}" for doc in retrieved_docs]
    )

    return result_text


#### Step 6: Initialize the ADK Agent
Finally, you instantiate an `LlmAgent` from the ADK. You provide it with a Gemini model (e.g., `gemini-2.0-flash`), a name, and importantly, the `faiss_search` function as one of its `tools`. The `instruction` is crucial as it guides the agent on when and how to use the `faiss_search` tool, enabling the RAG pattern.

In [ ]:
from google.adk.agents import LlmAgent
agent = LlmAgent(
    model="gemini-2.0-flash",
    name="rag_agent",
    tools=[faiss_search],
    instruction="You are a helpful assistant. If the user asks a question that requires knowledge about large language models, vector databases, or AI development, use the 'faiss_search' tool to find relevant information before answering and only use 1 context. If the user's question is general, you can answer directly."
)

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

APP_NAME = "APP_DOCUMENT" # New App Name
USER_ID = "user01"
SESSION_ID_BASE = "session01"

async def session_declaration():
    session_service = InMemorySessionService()
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID_BASE
    )
    runner = Runner(
        agent=agent, # The agent we want to run
        app_name=APP_NAME,   # Associates runs with our app
        session_service=session_service # Uses our session manager
    )
    return session, runner

The `call_agent_async` function serves as the primary interface for sending a user query to the previously defined `rag_agent` and processing its responses within the Agent Development Kit (ADK) framework.


In [ ]:
async def call_agent_async(query: str):
    sess, run = await session_declaration()
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response." # Default

    # Key Concept: run_async executes the agent logic and yields Events.
    # We iterate through events to find the final answer.
    async for event in run.run_async(user_id=USER_ID, session_id=SESSION_ID_BASE, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                print(event.content.parts)
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            # Add more checks here if needed (e.g., specific error codes)
            break # Stop processing events once the final response is found
    #print(final_response_text)
    return final_response_text

In [ ]:
result = await call_agent_async('What is ADK?')
print("-------------------------------------------")
print("Agent >>> ", result)


>>> User Query: What is ADK?


  [Event] Author: rag_agent, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'k': 1,
      'query': 'What is ADK?'
    },
    id='adk-7ae9b431-2de9-4d45-907c-62f9a54d98a4',
    name='faiss_search'
  )
)] role='model'


  [Event] Author: rag_agent, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-7ae9b431-2de9-4d45-907c-62f9a54d98a4',
    name='faiss_search',
    response={
      'result': """Retrieved documents:
- The Agent Development Kit (ADK) simplifies building multi-agent applications."""
    }
  )
)] role='user'
  [Event] Author: rag_agent, Type: Event, Final: True, Content: parts=[Part(
  text="""The Agent Development Kit (ADK) simplifies building multi-agent applications.
"""
)] role='model'
[Part(
  text="""The Agent Development Kit (ADK) simplifies building multi-agent applications.
"""
)]
-------------------------------------------
Agent >>>  The Agent Development Kit (ADK) simplifies building multi-agent applications.

